In [5]:
import pandas as pd

# تحميل البيانات مع تحديد أسماء الأعمدة
df = pd.read_csv("Data/Tweets.txt", sep=None, engine="python", on_bad_lines='skip', header=None, names=["text", "label"])  # غير الاسم حسب ملفك

# حذف NEUTRAL و OBJ
df = df[~df["label"].str.lower().isin(["neutral", "obj"])]

# قوائم كلمات مفتاحية لكل حالة
anxiety_keywords = ["قلق", "توتر", "خايف", "مش مرتاح", "نرفوزي", "أتوتر", "وسواس"]
depression_keywords = ["اكتئاب", "محبط", "تعبان", "مش قادر", "مش طايق", "يأس", "مفيش فايدة", "بائس"]
stress_keywords = ["ضغط", "مسؤوليات", "مشغول", "زهقان", "كله عليا", "حمل كبير"]
normal_keywords = ["سعيد", "مرتاح", "مبسوط", "فخور", "تمام", "كويس", "متحمس"]

def map_label(text, original_label):
    text = str(text).lower()
    label = original_label.lower()

    if label == "pos":
        return "Normal"
    elif label == "neg":
        if any(word in text for word in anxiety_keywords):
            return "Anxiety"
        elif any(word in text for word in depression_keywords):
            return "Depression"
        elif any(word in text for word in stress_keywords):
            return "Stress"
        else:
            return "Anxiety"  # أو "Unknown" حسب رغبتك
    else:
        return None  # لن تظهر لأننا حذفناهم بالفعل

# تطبيق التعديل على الليبل
df["new_label"] = df.apply(lambda row: map_label(row["text"], row["label"]), axis=1)

# حذف أي صف لم يتم تصنيفه (احتياطيًا)
df = df[df["new_label"].notnull()]

# حفظ الملف الجديد
df.to_csv("cleaned_dataset.csv", index=False)

print("✅ تم تحويل الليبلز وحذف NEUTRAL/OBJ وحفظ الملف كـ cleaned_dataset.csv")

✅ تم تحويل الليبلز وحذف NEUTRAL/OBJ وحفظ الملف كـ cleaned_dataset.csv


In [2]:
import pandas as pd

# قراءة الملف
df = pd.read_excel("Data/final_data.xlsx", engine='openpyxl')

# عرض كل القيم الفريدة في عمود label
print(df['label'].unique())

['اكتئاب' 'ميول انتحاريه' 'القلق']


In [4]:

# دالة الترجمة
def translate_label(label):
    label = str(label).strip().lower()
    if label in ["القلق"]:
        return "Anxiety"
    elif label in ["اكتئاب"]:
        return "Depression"
    elif label in ["ميول انتحاريه"]:
        return "Suicide"
    else:
        return None

# تطبيق الترجمة
df["translated_label"] = df["label"].apply(translate_label)

# حذف الصفوف التي لم يتم تصنيفها
df = df[df["translated_label"].notnull()]

# حفظ الملف الجديد
df.to_excel("final_data_translated.xlsx", index=False)

print("✅ تم ترجمة الليبلز وحفظ الملف كـ final_data_translated.xlsx")

✅ تم ترجمة الليبلز وحفظ الملف كـ final_data_translated.xlsx


In [10]:
import pandas as pd

# قراءة ملف cleaned_dataset.csv
df1 = pd.read_csv("cleaned_dataset.csv")

# قراءة ملف final_data.xlsx
df2 = pd.read_excel("Data/final_data.xlsx", engine='openpyxl')

# توحيد أسماء الأعمدة إذا لزم الأمر
df2 = df2.rename(columns={"translated_label": "new_label"}) if "translated_label" in df2.columns else df2

# إذا كان في df2 عمود label يحتاج ترجمة، طبق دالة الترجمة
if "new_label" not in df2.columns and "label" in df2.columns:
    def translate_label(label):
        label = str(label).strip().lower()
        if label in ["القلق"]:
            return "Anxiety"
        elif label in ["اكتئاب"]:
            return "Depression"
        elif label in ["ميول انتحاريه"]:
            return "Suicide"
        elif label in ["ضغط"]:
            return "Stress"
        elif label in ["ايجابي", "سعيد", "مرتاح", "مبسوط", "فخور", "تمام", "كويس", "متحمس"]:
            return "Normal"
        else:
            return None
    df2["new_label"] = df2["label"].apply(translate_label)
    df2 = df2[df2["new_label"].notnull()]

# التأكد من وجود أعمدة موحدة: text, new_label
df1 = df1[["text", "new_label"]]
df2 = df2[["text", "new_label"]]

# دمج الملفين
combined_df = pd.concat([df1, df2], ignore_index=True)

# حفظ الملف المدمج
combined_df.to_csv("combined_dataset.csv", index=False)


In [13]:
df = pd.read_csv("combined_dataset.csv")
# show columns
print(df.columns)
# show unique values in new_label
print(df['new_label'].unique())
#show the size of the dataset
print(f"Size of the dataset: {df.shape[0]} rows")

Index(['text', 'new_label'], dtype='object')
['Normal' 'Anxiety' 'Depression' 'Stress' 'Suicide']
Size of the dataset: 4197 rows


In [17]:
#read all_arabic.csv and drop text column and rename arabic_text to text
df_arabic = pd.read_csv("all_arabic.csv")
df_arabic = df_arabic.drop(columns=["text"]).rename(columns={"arabic_text": "text"})
df_arabic = df_arabic.drop(columns=["arabic_label"])
# save the dataframe to csv
df_arabic.to_csv("all_arabic.csv", index=False)


In [18]:
import pandas as pd

# قراءة ملف combined_dataset.csv
df1 = pd.read_csv("combined_dataset.csv")

# قراءة ملف all_arabic.csv
df2 = pd.read_csv("all_arabic.csv")

# التأكد من وجود الأعمدة المطلوبة
if "new_label" not in df2.columns and "label" in df2.columns:
    # إذا كان هناك عمود label فقط، انسخه إلى new_label
    df2["new_label"] = df2["label"]

# التأكد من وجود عمود text
df1 = df1[["text", "new_label"]]
df2 = df2[["text", "new_label"]]

# دمج الملفين
final_df = pd.concat([df1, df2], ignore_index=True)

# حفظ الملف النهائي
final_df.to_csv("final_combined_dataset.csv", index=False)

print("✅ تم دمج combined_dataset.csv مع all_arabic.csv وحفظ الناتج في final_combined_dataset.csv")

✅ تم دمج combined_dataset.csv مع all_arabic.csv وحفظ الناتج في final_combined_dataset.csv


In [24]:
df = pd.read_csv("final_combined_dataset.csv")
# عرض الأعمدة
print(df.columns)
# عرض القيم الفريدة في عمود new_label
print(df['new_label'].unique())
# عرض حجم البيانات
print(f"Size of the dataset: {df.shape[0]} rows")
# show the first 5 rows
print(df.head())

Index(['text', 'new_label'], dtype='object')
['Normal' 'Anxiety' 'Depression' 'Stress' 'Suicide']
Size of the dataset: 18787 rows
                                                text new_label
0  أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...    Normal
1  البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...   Anxiety
2  امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #...    Normal
3  عودة جماعة الإخوان إلى الحياة السياسية بنفس وض...   Anxiety
4              الصداقة تزرع الحياة أزهارًا #مي_زيادة    Normal
